In [9]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import pickle

root_dir = '/om/user/ckauf/neural-nlp/neural_nlp/analyze/neural-scrambled/metric-validation'
stimuli_path = os.path.join(root_dir,'indep_stimuli_dfs')
print(stimuli_path)
conditions = ['Original', 'Scr1', 'Scr3', 'Scr5', 'Scr7', 'lowPMI', 'backward', 'random']

/om/user/ckauf/neural-nlp/neural_nlp/analyze/neural-scrambled/metric-validation/indep_stimuli_dfs


In [10]:
#look into df structure
def look_at_df(stimuli_path):
    stimuli_dict = {}
    for filename in os.listdir(stimuli_path):
        if 'Original' in filename:
            with open(os.path.join(stimuli_path,filename), "rb") as f:
                condition = filename.split("_")[-1].split(".")[0]
                df = pickle.load(f)
    return df

In [11]:
df = look_at_df(stimuli_path)
df.head()

,sentence,sentence_num,stimulus_id,experiment,story,passage_index,passage_label,passage_category
0,beekeeping encourages the conservation of loca...,0,243sentences.0,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
1,it is in every beekeeper's interest to conserv...,1,243sentences.1,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
2,as a passive form of agriculture it does not r...,2,243sentences.2,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
3,beekeepers also discourage the use of pesticid...,3,243sentences.3,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
4,artisanal beekeepers go to extremes for their ...,4,243sentences.4,243sentences,243sentences.beekeeping,2,beekeeping,beekeeping


In [12]:
stimulus_ids = list(df['stimulus_id'])

# prepare stimuli lists for shuffling

In [13]:
def get_stimuli_dict(stimuli_path):
    stimuli_dict = {}
    for cond in conditions:
        for filename in os.listdir(stimuli_path):
            if cond in filename:
                with open(os.path.join(stimuli_path,filename), "rb") as f:
                    condition = filename.split("_")[-1].split(".")[0]
                    df = pickle.load(f)
                    stimuli = list(df['sentence'])
                    stimulus_ids = list(df['stimulus_id'])
                    stimuli_dict[condition] = list(zip(stimulus_ids, stimuli))
    return stimuli_dict

In [14]:
stimuli_dict = get_stimuli_dict(stimuli_path)
stimuli_dict

{'Original': [('243sentences.0',
   'beekeeping encourages the conservation of local habitats'),
  ('243sentences.1',
   "it is in every beekeeper's interest to conserve local plants that produce pollen"),
  ('243sentences.2',
   'as a passive form of agriculture it does not require that native vegetation be cleared to make way for crops'),
  ('243sentences.3',
   'beekeepers also discourage the use of pesticides on crops because they could kill the honeybees'),
  ('243sentences.4',
   'artisanal beekeepers go to extremes for their craft but their product is worth the effort'),
  ('243sentences.5',
   'artisanal honey-making emphasizes quality and character over quantity and consistency'),
  ('243sentences.6',
   'to produce the finest honey beekeepers become micromanagers of their honeybees'),
  ('243sentences.7',
   'they scout the fields know when nectar flows and select the best ways to extract honey'),
  ('243sentences.8',
   'as the beekeeper opens the hive the deep hum of 40000 

In [15]:
stimuli_dict.keys()

dict_keys(['Original', 'Scr1', 'Scr3', 'Scr5', 'Scr7', 'lowPMI', 'backward', 'random'])

# For each sentence prepare 10 random orderings and assert that they are not in any of the other lists

In [16]:
import random

#generate 10 random permutations of each sequence
def perm_generator(seq, seen):
    seen = set(seen)
    #print(len(seen))
    length = len(seq)
    while True:
        perm = tuple(random.sample(seq, length))
        if perm not in seen:
            seen.add(perm)
            #print(len(seen))
            yield perm
        else:
            print('Already saw this permutation!')
        #print('*'*30)

In [17]:
N = 10 #number of desired shufflings
def get_10_random_perms(stimuli_dict, N):

    ten_random_permutations_dict = {}

    for ind, (identifier, elm) in enumerate(stimuli_dict['Original']):
        
        print(ind, identifier, elm)
        words = elm.split()

        #add permutations of other conditions of this sentence to the "seen" set s.th. the generated
        #permutations do not include already seen shuffling conditions
        seen = [tuple(stimuli_dict[cond][ind][1].split()) for cond in conditions]
        rand_perms = perm_generator(words, seen)
        perms = [next(rand_perms) for _ in range(N)]
        perms = [list(elm) for elm in perms]
        
        #additional check for mutual exclusivity
        scr_conds = [list(elm) for elm in seen]
        for elm in scr_conds:
            assert elm not in perms
        for elm in perms:
            assert elm not in scr_conds
            
        perms = [' '.join(elm) for elm in perms]
            
        print(perms)
        print('\n')
        ten_random_permutations_dict[identifier] = perms
        
    return ten_random_permutations_dict

In [18]:
ten_random_permutations_dict = get_10_random_perms(stimuli_dict, N)

0 243sentences.0 beekeeping encourages the conservation of local habitats
['habitats conservation encourages the beekeeping of local', 'beekeeping the of local encourages habitats conservation', 'of habitats conservation beekeeping the local encourages', 'habitats encourages local beekeeping the conservation of', 'encourages conservation local of the beekeeping habitats', 'beekeeping local habitats encourages conservation the of', 'habitats of conservation local beekeeping encourages the', 'encourages habitats local conservation beekeeping of the', 'the beekeeping local conservation of habitats encourages', 'the conservation habitats local of beekeeping encourages']


1 243sentences.1 it is in every beekeeper's interest to conserve local plants that produce pollen
["to interest beekeeper's that conserve local produce every pollen in is plants it", "in is every interest it that conserve pollen produce beekeeper's local plants to", "to produce interest it plants every pollen is local con

["the to loss of adjust patient's have to independence families", "loss the adjust have independence patient's to of to families", "families loss to independence the have of patient's adjust to", "independence loss to families adjust have patient's the of to", "loss the of to families have to independence adjust patient's", "the have of to loss families patient's adjust independence to", "of loss have to to patient's adjust independence the families", "loss adjust families independence the have to of to patient's", "loss the patient's adjust of families to independence have to", "have adjust to patient's loss to independence of the families"]


143 243sentences.143 and for society hip fractures involve substantial expense and time in the hospital
['fractures substantial hospital and for hip and involve society in time the expense', 'fractures for society substantial the hospital and expense hip and involve in time', 'for substantial fractures and society hip involve time hospital the e

['blade container the have a with bottom glass blenders or plastic rotating in a', 'bottom container or a a have blade in plastic with rotating the glass blenders', 'with blenders glass bottom plastic a container rotating the in or have blade a', 'container rotating with a have in or a the blenders bottom blade plastic glass', 'blenders a glass or bottom with the have a in plastic rotating blade container', 'bottom plastic a glass or container blade blenders have a the in with rotating', 'blade or glass with a a bottom blenders plastic container in have the rotating', 'a in blenders have blade with rotating or glass a plastic container bottom the', 'or the have rotating with container a glass in a blenders blade bottom plastic', 'plastic the container blade blenders have or glass a rotating in with a bottom']


289 384sentences.46 blenders are used to make milk shakes and fruit smoothies
['fruit are and used shakes blenders smoothies make milk to', 'used shakes fruit are and milk make 

['can attack stabbing throwing knife by to be used or slashing a', 'a be or throwing to by stabbing slashing can knife attack used', 'a be throwing slashing by can attack to or knife used stabbing', 'to or a used stabbing slashing knife can by attack throwing be', 'knife attack stabbing used throwing be by a can slashing to or', 'slashing attack can used throwing a be knife stabbing or to by', 'can attack knife by throwing to stabbing used slashing a be or', 'can to be stabbing used throwing by attack knife a or slashing', 'by knife can throwing to stabbing attack slashing or be a used', 'attack knife slashing by used a or be stabbing throwing can to']


435 384sentences.192 lentils are the seeds of a plant of the legume family
['the family of legume of the are plant seeds lentils a', 'plant seeds are of the the a legume of family lentils', 'plant of are legume a lentils of the seeds the family', 'seeds plant of family the of the a legume lentils are', 'family a the legume of are the o


586 384sentences.343 there are many varieties of tomato with different sizes and tastes
['many varieties with tastes there different sizes of and tomato are', 'and with tomato many tastes are there varieties different of sizes', 'many of tastes sizes and tomato different there varieties are with', 'sizes many there with different and of tomato varieties are tastes', 'many sizes tomato different of varieties with tastes and are there', 'of sizes tomato varieties there and many different are tastes with', 'different of many and with sizes varieties tomato are tastes there', 'with are sizes tastes and tomato different of varieties many there', 'are and many sizes varieties different tastes with tomato there of', 'with of many sizes tomato varieties and there are different tastes']


587 384sentences.344 a train is a connected series of cars that moves between stations on a track
['is a between track of stations that on a moves a cars train connected series', 'series connected train that 

In [19]:
print(list(ten_random_permutations_dict.keys())[:10])
print(len(ten_random_permutations_dict['243sentences.0']))
print(ten_random_permutations_dict['243sentences.0'][:10][0])

['243sentences.0', '243sentences.1', '243sentences.2', '243sentences.3', '243sentences.4', '243sentences.5', '243sentences.6', '243sentences.7', '243sentences.8', '243sentences.9']
10
habitats conservation encourages the beekeeping of local


# let's investigate the Mollica pPMI outputs again

In [35]:
os.chdir('/om2/user/ckauf/Mollica2020Stimuli/Calculate_PMI')
pPMI1_M = pd.read_csv('pPMI_1.csv',header=None)
pPMI1_M.head()

,0,1,2,3
0,Experiment1_S_1,Intact,on their last day they were overwhelmed by far...,39.514989
1,Experiment1_S_2,Intact,the detective looked from one man to another a...,36.516106
2,Experiment1_S_3,Intact,he dismissed reports that his party was divide...,36.003851
3,Experiment1_S_4,Intact,the registration of students and all grants an...,39.890960
4,Experiment1_S_5,Intact,the small audience had begun to fidget on thei...,46.837785


In [36]:
os.chdir('/om2/user/ckauf/Mollica2020Stimuli')
sentences = []
sample = []
with open('stimuli.csv') as f:
    for line in f.readlines():
        entry = line.split(',')
        print(entry)
        sentences.append(entry)
        sample.append(entry[2])
        print('Sentence: ', sentences)
        print('Sample: ',sample)
        break

['Experiment1_S_1', 'Intact', 'on their last day they were overwhelmed by farewell messages and gifts\n']
Sentence:  [['Experiment1_S_1', 'Intact', 'on their last day they were overwhelmed by farewell messages and gifts\n']]
Sample:  ['on their last day they were overwhelmed by farewell messages and gifts\n']


In [20]:
os.chdir('/om2/user/ckauf/Mollica2020Stimuli/Calculate_PMI')
pPMI1 = pd.read_csv('pPMI_1_6884.csv',header=None)

In [21]:
pPMI1

,0,1,2,3
0,243sentences.0,Original,beekeeping encourages the conservation of loca...,24.961928
1,243sentences.1,Original,it is in every beekeeper's interest to conserv...,36.307457
2,243sentences.2,Original,as a passive form of agriculture it does not r...,80.396020
3,243sentences.3,Original,beekeepers also discourage the use of pesticid...,53.453739
4,243sentences.4,Original,artisanal beekeepers go to extremes for their ...,40.724573
...,...,...,...,...
4384,384sentences.379,random,to graphics long and any beekeepers and is a,9.862115
4385,384sentences.380,random,ripe a after it teflon-clad barrel through,7.679931
4386,384sentences.381,random,a scholarly write patience pharmacists played ...,21.221625
4387,384sentences.382,random,and less others clouds afraid food-scarce and are,5.410695


In [22]:
stimuli6884 = pd.read_csv(os.path.join('/om2/user/ckauf/Mollica2020Stimuli','stimuli_6.884.csv'),header=None)
stimuli6884

,0,1,2
0,243sentences.0,Original,beekeeping encourages the conservation of loca...
1,243sentences.1,Original,it is in every beekeeper's interest to conserv...
2,243sentences.2,Original,as a passive form of agriculture it does not r...
3,243sentences.3,Original,beekeepers also discourage the use of pesticid...
4,243sentences.4,Original,artisanal beekeepers go to extremes for their ...
...,...,...,...
4384,384sentences.379,random,to graphics long and any beekeepers and is a
4385,384sentences.380,random,ripe a after it teflon-clad barrel through
4386,384sentences.381,random,a scholarly write patience pharmacists played ...
4387,384sentences.382,random,and less others clouds afraid food-scarce and are


# OK so let's create a similar df for our randomly permuted stimuli

In [37]:
os.chdir(os.path.join(root_dir, 'add_random_lowPMI_condition'))
list(ten_random_permutations_dict.keys())[:10]

['243sentences.0',
 '243sentences.1',
 '243sentences.2',
 '243sentences.3',
 '243sentences.4',
 '243sentences.5',
 '243sentences.6',
 '243sentences.7',
 '243sentences.8',
 '243sentences.9']

In [38]:
def flatten_array(liste):
    liste_flatten = [item for sublist in liste for item in sublist]
    return liste_flatten

In [39]:
stimulus_ids_lst = flatten_array([[elm] * 10 for elm in list(ten_random_permutations_dict.keys())])
sentence_lst = flatten_array(list(ten_random_permutations_dict.values()))
print(len(stimulus_ids_lst), len(sentence_lst))
print(stimulus_ids_lst[:20])
print(sentence_lst[:20])

6270 6270
['243sentences.0', '243sentences.0', '243sentences.0', '243sentences.0', '243sentences.0', '243sentences.0', '243sentences.0', '243sentences.0', '243sentences.0', '243sentences.0', '243sentences.1', '243sentences.1', '243sentences.1', '243sentences.1', '243sentences.1', '243sentences.1', '243sentences.1', '243sentences.1', '243sentences.1', '243sentences.1']
['habitats conservation encourages the beekeeping of local', 'beekeeping the of local encourages habitats conservation', 'of habitats conservation beekeeping the local encourages', 'habitats encourages local beekeeping the conservation of', 'encourages conservation local of the beekeeping habitats', 'beekeeping local habitats encourages conservation the of', 'habitats of conservation local beekeeping encourages the', 'encourages habitats local conservation beekeeping of the', 'the beekeeping local conservation of habitats encourages', 'the conservation habitats local of beekeeping encourages', "to interest beekeeper's tha

In [40]:
def get_10_random_perms_df(ten_random_permutations_dict):
    stimulus_ids_lst = flatten_array([[elm] * 10 for elm in list(ten_random_permutations_dict.keys())])
    sentence_lst = flatten_array(list(ten_random_permutations_dict.values()))
    
    df = pd.DataFrame(
        {'stimulus_id' : stimulus_ids_lst,
        'permutated_stimulus' : sentence_lst})
    
    return df

In [41]:
random_df = get_10_random_perms_df(ten_random_permutations_dict)
random_df

,stimulus_id,permutated_stimulus
0,243sentences.0,habitats conservation encourages the beekeepin...
1,243sentences.0,beekeeping the of local encourages habitats co...
2,243sentences.0,of habitats conservation beekeeping the local ...
3,243sentences.0,habitats encourages local beekeeping the conse...
4,243sentences.0,encourages conservation local of the beekeepin...
...,...,...
6265,384sentences.383,a than has organs different reproductive woman...
6266,384sentences.383,different a woman reproductive a has than orga...
6267,384sentences.383,man woman different than reproductive a a orga...
6268,384sentences.383,has organs than different a man woman reproduc...


In [42]:
test_df = random_df.loc[random_df['stimulus_id'] == '243sentences.0']
test_df

,stimulus_id,permutated_stimulus
0,243sentences.0,habitats conservation encourages the beekeepin...
1,243sentences.0,beekeeping the of local encourages habitats co...
2,243sentences.0,of habitats conservation beekeeping the local ...
3,243sentences.0,habitats encourages local beekeeping the conse...
4,243sentences.0,encourages conservation local of the beekeepin...
5,243sentences.0,beekeeping local habitats encourages conservat...
6,243sentences.0,habitats of conservation local beekeeping enco...
7,243sentences.0,encourages habitats local conservation beekeep...
8,243sentences.0,the beekeeping local conservation of habitats ...
9,243sentences.0,the conservation habitats local of beekeeping ...


In [49]:
#test_df.to_csv('TEST_into_calculatePMI_10randompermutations.csv',header=False)
random_df.to_csv('into_calculatePMI_10randompermutations.csv',header=False)

In [44]:
import pickle

sentences = []
sample = []
with open('TEST_into_calculatePMI_10randompermutations.csv') as f:
    for line in f.readlines():
        entry = line.split(',')
        print(entry)
        sentences.append(entry)
        sample.append(entry[2])
        print('Sentence: ', sentences)
        print('Sample: ',sample)
        break

['0', '243sentences.0', 'habitats conservation encourages the beekeeping of local\n']
Sentence:  [['0', '243sentences.0', 'habitats conservation encourages the beekeeping of local\n']]
Sample:  ['habitats conservation encourages the beekeeping of local\n']


# How to run afterwards

Change input file in GrabNGrams.py & pPMI.py!

* module load openmind/miniconda/4.0.5-python3
* srun -t 01:00:00 GrabNGrams.py > grabngrams_out.txt
* srun -t 01:00:00 pPMI.py > ppmi_out.txt